In [ ]:
"""
在 PyTorch 中，张量的形状操作是非常重要的，因为它允许你灵活地调整张量的维度和结构，以适应不同的计算需求。
一般我们会用以下两种方法来操作张量的形状：

    view()：
        返回的是原始张量视图，不重新分配内存，效率更高
        高效，但需要张量在内存中是连续的(如果不连续会报错)

    reshape()：
        可以用于将张量转换为不同的形状，但要确保转换后的形状与原始形状具有相同的元素数量。
        更灵活，但涉及内存复制，效率较低

        
可以使用 Tensor.is_contiguous() 检查张量在内存中是否连续存储
    无需传参，该方法的返回值为布尔值
可以使用 Tensor.contiguous() 创建连续副本
    无需传参，该方法返回一个连续副本
"""

In [4]:
"""view()方法"""

import torch

tensor = torch.tensor([[1, 2, 3], [4, 5, 6]])
print("正常情况下的张量：", tensor.is_contiguous())
print('进行view改变形状：\n', tensor.view(3, 2))

# 对张量进行转置操作
tensor = tensor.t()
print("转置操作的张量：", tensor.is_contiguous())
print(tensor)
# 此时使用view进行变形操作
tensor = tensor.view(2, -1)
print(tensor)


正常情况下的张量： True
进行view改变形状：
 tensor([[1, 2],
        [3, 4],
        [5, 6]])
转置操作的张量： False
tensor([[1, 4],
        [2, 5],
        [3, 6]])


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [ ]:
"""reshape()方法"""

import torch

data = torch.randint(0, 10, (4, 3))
print(data)
# 1. 使用reshape改变形状
data = data.reshape(2, 2, 3)
print(data)

# 2. 使用-1表示自动计算
data = data.reshape(2, -1)
print(data)



In [ ]:
"""
交换张量的维度：
    torch.transpose(tensor, dim0, dim1)
        用于交换张量的两个维度，返回原张量的视图(view)，不复制数据。
        注意：仅能交换两个维度，适用于简单转置操作。
        底层：通过调整stride实现，时间复杂度O(1)。

    tensor.permute(*dims)  *常用*
        重新排列所有维度的顺序，返回新视图(view)，不复制数据。
        特点：可以一次性重排多个维度，更灵活。
        底层：同样通过调整stride实现，时间复杂度O(1)。
        
    共同点：
        1. 都是视图操作(view)，不复制数据
        2. 都返回非连续(non-contiguous)张量
        3. 如果后续需要连续内存，需调用.contiguous()    
"""

In [2]:
"""
torch.transpose 和 torch.permute 示例
"""

import torch

# 创建原始张量 (3D: 批次×高度×宽度)
original = torch.arange(24).reshape(2, 3, 4)  # 形状 [2, 3, 4]
print("=== 原始张量 ===")
print(f"形状: {original.shape}")
print(f"步长: {original.stride()}")  # (12, 4, 1)
print(f"连续: {original.is_contiguous()}")
print("数据:\n", original)

# 使用transpose交换维度0和2 (批次和宽度)
transposed = torch.transpose(original, 0, 2)  # 形状变为 [4, 3, 2]
print("\n=== transpose(0,2)结果 ===")
print(f"形状: {transposed.shape}")
print(f"步长: {transposed.stride()}")  # (1, 4, 12) - 步长改变
print(f"连续: {transposed.is_contiguous()}")  # False
print(f"共享数据: {transposed.storage().data_ptr() == original.storage().data_ptr()}")  # True
print("转置后数据:\n", transposed)

# 使用permute重排所有维度 (新顺序: 宽度×高度×批次)
permuted = original.permute(2, 1, 0)  # 形状变为 [4, 3, 2]
print("\n=== permute(2,1,0)结果 ===")
print(f"形状: {permuted.shape}")
print(f"步长: {permuted.stride()}")  # (1, 4, 12) - 与transposed相同
print(f"连续: {permuted.is_contiguous()}")  # False
print(f"共享数据: {permuted.storage().data_ptr() == original.storage().data_ptr()}")  # True

# 验证transpose和permute结果是否相同
print("\n=== 比较结果 ===")
print(f"transposed和permuted形状相同: {transposed.shape == permuted.shape}")
print(f"transposed和permuted数据相等: {torch.equal(transposed, permuted)}")  # True

# 连续化操作的影响
print("\n=== 连续化操作 ===")
contiguous_transposed = transposed.contiguous()
print(f"连续化后内存地址: {contiguous_transposed.storage().data_ptr() != original.storage().data_ptr()}")  # True
print(f"连续化后是否连续: {contiguous_transposed.is_contiguous()}")  # True

# 更复杂的permute示例 (重排多个维度)
print("\n=== 复杂permute示例 ===")
# 原始形状 [2,3,4] -> 新顺序 [2,0,1] 解释:
# 新维度0 = 原维度2 (最内层)
# 新维度1 = 原维度0 (批次)
# 新维度2 = 原维度1 (高度)
complex_perm = original.permute(2, 0, 1)  # 形状 [4,2,3]
print(f"新形状: {complex_perm.shape}")  # [4,2,3]
print(f"新步长: {complex_perm.stride()}")  # (1,12,4)
print("重排后数据:\n", complex_perm)

# 尝试在非连续张量上使用view (会报错)
try:
    print("\n=== 在非连续张量上使用view ===")
    transposed.view(4, 6)  # 会报错
except RuntimeError as e:
    print(f"错误信息: {e}")
    print("解决方案: 先调用.contiguous()")
    fixed = transposed.contiguous().view(4, 6)
    print("修复后:\n", fixed)

=== 原始张量 ===
形状: torch.Size([2, 3, 4])
步长: (12, 4, 1)
连续: True
数据:
 tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])

=== transpose(0,2)结果 ===
形状: torch.Size([4, 3, 2])
步长: (1, 4, 12)
连续: False
共享数据: True
转置后数据:
 tensor([[[ 0, 12],
         [ 4, 16],
         [ 8, 20]],

        [[ 1, 13],
         [ 5, 17],
         [ 9, 21]],

        [[ 2, 14],
         [ 6, 18],
         [10, 22]],

        [[ 3, 15],
         [ 7, 19],
         [11, 23]]])

=== permute(2,1,0)结果 ===
形状: torch.Size([4, 3, 2])
步长: (1, 4, 12)
连续: False
共享数据: True

=== 比较结果 ===
transposed和permuted形状相同: True
transposed和permuted数据相等: True

=== 连续化操作 ===
连续化后内存地址: True
连续化后是否连续: True

=== 复杂permute示例 ===
新形状: torch.Size([4, 2, 3])
新步长: (1, 12, 4)
重排后数据:
 tensor([[[ 0,  4,  8],
         [12, 16, 20]],

        [[ 1,  5,  9],
         [13, 17, 21]],

        [[ 2,  6, 10],
         [14, 18, 22]],

        [[ 

C:\Users\26595\AppData\Local\Temp\ipykernel_22820\162467243.py:21: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  print(f"共享数据: {transposed.storage().data_ptr() == original.storage().data_ptr()}")  # True


In [ ]:
"""
在后续的网络学习中，升维和降维是常用操作，需要掌握。

    Tensor.unsqueeze(dim: int)
        用于在指定维度前插入一个大小为 1 的新维度。
        (如果dim=-1，则会将新维度插入在末尾)
    Tensor.squeeze(dim=None)
        用于移除所有大小为 1 的维度，或者移除指定维度的大小为 1 的维度。
"""

In [ ]:
"""
Tensor.unsqueeze 和 Tensor.squeeze 方法示例
"""

import torch

# 原始 2D 张量 (3x4 矩阵)
t = torch.tensor([[1, 2, 3, 4],
                 [5, 6, 7, 8],
                 [9, 10, 11, 12]])
print("原始形状:", t.shape)  # torch.Size([3, 4])

# ========== 升维操作 (unsqueeze) ==========
# 在维度0插入新维度 (最外层)
t1 = t.unsqueeze(0)
print("\nunsqueeze(0) 后:", t1.shape)  # torch.Size([1, 3, 4])

# 在维度1插入新维度 (行与列之间)
t2 = t.unsqueeze(1)
print("unsqueeze(1) 后:", t2.shape)  # torch.Size([3, 1, 4])

# 在最后插入新维度 (dim=-1)
t3 = t.unsqueeze(-1)
print("unsqueeze(-1) 后:", t3.shape)  # torch.Size([3, 4, 1])

# ========== 降维操作 (squeeze) ==========
# 创建含单值维度的张量
t4 = torch.zeros(2, 1, 3, 1, 4)  # 形状: [2,1,3,1,4]
print("\n原始含单值维度:", t4.shape)

# 默认移除所有单值维度
t5 = t4.squeeze()
print("squeeze() 后:", t5.shape)  # torch.Size([2, 3, 4])

# 仅移除指定维度 (dim=1)
t6 = t4.squeeze(1)
print("squeeze(dim=1) 后:", t6.shape)  # torch.Size([2, 3, 1, 4])

# 尝试移除非单值维度 (无变化)
t7 = t5.squeeze(0)  # 第0维是2(非1)
print("尝试移除非单值维度:", t7.shape)  # 仍为 torch.Size([2, 3, 4])

In [ ]:
"""Tensor也有广播机制，与numpy相同"""